In [1]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
from itertools import combinations

In [2]:
#Veriler, tarih ve ürün adları ile okunuyor ve birleştiriliyor
path_list=io.get_path('data')
cols= ["Time","BID price","ASK price"]
master_data=pd.concat((pd.read_excel(file,usecols=cols).\
                       assign(name=info.get_productName(file),date=info.get_productDate(file)
                             ) for file in path_list)).reset_index(drop=True)
master_data.columns=['time','bid_price','ask_price','name','date']

In [3]:
#date sütunu Timestamp'e dönüştürülüyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')

In [4]:
#tarih ve saat bilgisi birleştiriliyor
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [5]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

In [6]:
# mid price hesaplanıyor
master_data['mid_price']=aux.get_mid_price(master_data)

In [7]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

In [8]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [19]:
c=list(pairs)

In [13]:
x=df_mid_price.reset_index()

In [15]:
g=x.groupby([x.date.dt.date,x.date.dt.hour])

In [108]:
v=('6BU8', '6CU8','date')

In [110]:
b= x.loc[:,v]

In [123]:
b

name,6BU8,6CU8,date
0,1.357685,0.71785,2018-09-06 00:00:00
1,1.357685,0.71785,2018-09-06 00:00:01
2,1.357685,0.71785,2018-09-06 00:00:02
3,1.357685,0.71785,2018-09-06 00:00:03
4,1.357794,0.71785,2018-09-06 00:00:04
...,...,...,...
79195,1.241450,0.71785,2018-09-07 23:59:55
79196,1.241450,0.71785,2018-09-07 23:59:56
79197,1.241450,0.71785,2018-09-07 23:59:57
79198,1.241450,0.71785,2018-09-07 23:59:58


In [122]:
b.groupby([b.date.dt.date,b.date.dt.hour]).apply(lambda x : find_spread(x,values))

ValueError: Key 2018-09-07 00:00:00 not in level Index([2018-09-06, 2018-09-07], dtype='object', name='date')

In [97]:
def pair_al(f_grp):
    l=[]
    tarih=f_grp.date
    for pair in c:
        df= f_grp.loc[:,pair].reset_index()
        print(df)
        spread=find_spread(df,values)
        pair_name='_'.join(pair)
        print(len(tarih),'-',len(spread))
        print(tarih)
        df_spread= pd.DataFrame({'tarih':tarih,pair_name:spread}).reset_index()
        
        l.append(df_spread)
    return pd.concat(l,axis=1)

In [62]:
def pair_al(f_grp):
    ex_dict={}
    for pair in c:
        ex_df=f_grp.loc[:,pair]
        ex_dict['_'.join(pair)]=find_spread(ex_df,values)
    return pd.DataFrame(ex_dict)

In [22]:
def find_spread(pair,values):
    """6A ve 6B verisinden SPREAD verisini üretir.
    Parameters:
        mid_price(tuple):
        values(dict): ticksize değerleri
    Returns:
        pd.Series: Hesaplanmış spread verisi
    """
    a_series=pair.iloc[:,0]
    b_series=pair.iloc[:,1]
    atick = values['a_PNLTICK']/values['a_TICKSIZE']
    btick = values['b_PNLTICK']/values['b_TICKSIZE']
    size = len(a_series)
    spread = size*[0]
    for i in range(size): 
        try:
            spread[i+1] = (((a_series.iat[i+1] - a_series.iat[i])*atick) - ((b_series.iat[i+1] - b_series.iat[i])*btick)) +spread[i] 
        except:            
            pass        
    return pd.Series(data=spread,index=a_series.index,name='spread')

In [25]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}